<div style="float:right"><i>Peter Norvig<br>5 January 2016</i></div>

# Countdown to 2016

In 2016 Alex Bellos [posed](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) this New Year's puzzle:


> Fill in the blanks so that this equation makes arithmetical sense:

> `10 _ 9 _ 8 _ 7 _ 6 _ 5 _ 4 _ 3 _ 2 _ 1 = 2016`

> You are allowed to use *only* the four basic arithmetical operations: +, -, &times;, ÷. But brackets (parentheses) can be used wherever needed. So, for example, the solution could begin

> `(10 - 9) / (8` ...

> or

> `10 + (9 - 8)` ...

Let's see if we can solve this puzzle, and some of the related ones from Alex's [first](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) and [second](http://www.theguardian.com/science/2016/jan/04/did-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) post.  We'll start with a  simpler version of the puzzle.

# Four Operators, No Brackets

Suppose for the moment we are not allowed to use brackets.  Then there are nine blanks,  each of which can be filled by one of four operators, so the total number of possible expressions is:

In [1]:
4 ** 9

262144

The function `itertools.product` can enumerate all the ways of filling 9 blanks with one of the four operators:

In [2]:
import itertools

operators = ('+', '-', '*', '/')
    
len(set(itertools.product(operators, repeat=9)))

262144

So we can fill in the equation with each possible sequence of operations, and evaluate each string to see if it equals 2016. But we need to catch errors such as dividing by zero, so we'll define a wrapper function, `evaluate`, to do the `eval`:

In [3]:
def evaluate(exp):
    "eval exp, or return None if there is an arithmetic error."
    try:
        return eval(exp)
    except ArithmeticError:
        return None

def solve_no_brackets(operators, target=2016):
    "All solutions to the countdown puzzle (with no brackets)."
    exps = ('10{}9{}8{}7{}6{}5{}4{}3{}2{}1'.format(*ops)
            for ops in itertools.product(operators, repeat=9))
    return [exp for exp in exps if evaluate(exp) == target]

solve_no_brackets(operators)

[]

Too bad; we did all that work and didn't find a solution. What years *can* we find solutions for? Let's modify `solve_no_brackets` to take a collection of target years rather than a single one, and return a dict of the form `{year: 'expression'}` for each expression that evaluates to one of the target years:

In [4]:
def solve_no_brackets(operators, targets):
    "All solutions to the countdown puzzle (with no brackets)."
    exps = ('10{}9{}8{}7{}6{}5{}4{}3{}2{}1'.format(*ops)
            for ops in itertools.product(operators, repeat=9))
    return {int(evaluate(exp)): exp for exp in exps if evaluate(exp) in targets}

solve_no_brackets(operators, range(1900, 2100))

{1979: '10*9*8+7*6*5*4*3/2-1',
 1980: '10*9*8+7*6*5*4*3/2/1',
 1981: '10*9*8+7*6*5*4*3/2+1',
 2013: '10*9*8*7/6/5*4*3-2-1',
 2014: '10*9*8*7/6/5*4*3-2/1',
 2015: '10*9*8*7/6/5*4*3-2+1',
 2017: '10*9*8*7/6/5*4*3+2-1',
 2018: '10*9*8*7/6/5*4*3+2/1',
 2019: '10*9*8*7/6/5*4*3+2+1'}

Interesting: in the 20th and 21st centuries, there are only two "golden eras" where the countdown equation works: the three year period centered on 1980, and the seven year period that is centered on 2016, but omits 2016. 

# Four Operators, With Brackets

Now let's return to the original puzzle, with the brackets. How many ways are there of bracketing an expression with 9 binary operators? I happen to remember that this is given by the [Catalan numbers](https://en.wikipedia.org/wiki/Catalan_number), and we can [look it up](http://www.wolframalpha.com/input/?i=9th+catalan+number) to find that there are 4862 diffferent bracketing. If we enumerated and evaluated all of them, it would take about 4862 times longer than doing a single `solve_no_brackets`, which took about 6 seconds, so the estimated time would be about 8 hours. I'm impatient, so I'd like a faster approach. 

I'll use the idea of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming): break the problem down into simpler subparts, and compute an answer for each subpart, and store intermediate results in a table so we don't need to re-compute them when we need them again.

How do we break the problem into parts? In general, any expression must consist of an operator with two operands (which might in turn be complex subexpressions). For example, a complete countdown expression might be of the form

    (10 ... 8) + (7 ... 1)
    
where `(10 ... 8)` means some expression that starts with 10 and ends with 8.  Of course we need not use `'+'` as the operator, and we need not split after the 8; we could use any of the four operators and split anywhere.  Let's start by defining `c10` as the tuple of integers forming the countdown from 10 to 1,  and the function `splits` to split a tuple in all ways: 

In [5]:
c10 = (10, 9, 8, 7, 6, 5, 4, 3, 2, 1)

def splits(items):
    "Split sequence of items into two non-empty parts, in all ways."
    return [(items[:i], items[i:]) 
            for i in range(1, len(items))]

In [6]:
splits(c10)

[((10,), (9, 8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9), (8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8), (7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7), (6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6), (5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5), (4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4), (3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3), (2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3, 2), (1,))]

Now what I would like to do is build up a table that says, for every subsequence of the numbers,  what are the expressions we can make with those numbers, and what do they evaluate to? We'll call the table `EXPS`. For example, with the subsequence `(10, 9, 8)`, we would have:

    EXPS[(10, 9, 8)] = {
      27: '((10+9)+8)',
      8:  '((10-9)*8)', 
      -7: '(10-(9+8))', 
      ...}
     
We'll do the same for every other subsequence, for example:

    EXPS[(7, 6, 5, 4, 3, 2, 1)] = {
      1:   '((((7/((6/5)-4))+3)*2)*1)',
      2:   '((((7/((6/5)-4))+3)*2)+1)',
      3.5: '((((7/((6/5)-4))+3)+2)+1)',
      4:   '((7-((6/5)*((4/3)+2)))+1)',
      ...}
     
Once we have the tables for these two subsequences, we can put them together to get the table for the complete  `countdown(10)` by considering all ways of taking a value from the first table, then one of the four operators, then a value from the second table.  For example, taking the first entry from each table, and the operator `'+'`, we would have:

    EXPS[(10, 9, 8, 7, 6, 5, 4, 3, 2, 1)] = { 
      28: '(((10+9)+8)+((((7/((6/5)-4))+3)*2)*1))',
      ...}


I can implement `EXPS` as a defaultdict of dicts, and define `expressions(numbers)` to fill in `EXPS` entries for `numbers` and all sub-sequences of `numbers`.  Within `expressions`, note that `Lnums` and `Rnums` are sequences of numbers, such as `(10, 9, 8)` and `(7, 6)`. `L` and `R` are numeric values, such as `27` and `1`. And `Lexp` and `Rexp` are strings, such as `"((10+9)+8)"` and `"(7-6)"`. Rather than catching division-by-zero errors, we just avoid the division when the denominator is 0.

In [7]:
from collections import defaultdict, Counter

EXPS = defaultdict(dict) # e.g., EXPS[(10, 9, 8)][27] == '((10+9)+8)'

def expressions(numbers):
    "Fill EXPS table for numbers, and all sub-sequences of numbers. Return EXPS[numbers]"
    if numbers in EXPS: # Already did the work
        pass
    elif len(numbers) == 1: # Only one way to make an expression out of a single number
        expr(numbers, numbers[0], str(numbers[0]))
    else: # Split in all ways; fill tables for left and right; combine tables in all ways
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in itertools.product(expressions(Lnums), expressions(Rnums)):
                Lexp, Rexp = '(' + EXPS[Lnums][L], EXPS[Rnums][R] + ')'
                expr(numbers, L * R, Lexp + '*' + Rexp)
                expr(numbers, L - R, Lexp + '-' + Rexp)
                expr(numbers, L + R, Lexp + '+' + Rexp)
                if R != 0: 
                    expr(numbers, L / R, Lexp + '/' + Rexp)
    return EXPS[numbers]

def expr(numbers, value, exp): 
    "Record exp as an expression with the given value, covering the sequence of numbers."
    EXPS[numbers][value] = exp

Let's give it a try:

In [8]:
expressions((10, 9, 8))

{-62: '(10-(9*8))',
 -7: '((10-9)-8)',
 -6.888888888888889: '((10/9)-8)',
 0.125: '((10-9)/8)',
 0.1388888888888889: '((10/9)/8)',
 0.5882352941176471: '(10/(9+8))',
 2.375: '((10+9)/8)',
 8: '((10-9)*8)',
 8.875: '(10-(9/8))',
 8.88888888888889: '((10/9)*8)',
 9: '((10-9)+8)',
 9.11111111111111: '((10/9)+8)',
 10: '(10/(9-8))',
 11: '((10+9)-8)',
 11.125: '(10+(9/8))',
 11.25: '((10*9)/8)',
 27: '((10+9)+8)',
 82: '((10*9)-8)',
 98: '((10*9)+8)',
 152: '((10+9)*8)',
 170: '(10*(9+8))',
 720: '((10*9)*8)'}

In [9]:
EXPS[(10, 9, 8)][27]

'((10+9)+8)'

That looks reasonable. Let's solve the whole puzzle.

# Countdown to 2016: A Solution

In [10]:
%time expressions(c10)[2016]

CPU times: user 29.5 s, sys: 698 ms, total: 30.2 s
Wall time: 30.2 s


'(((((((10+((9*8)*7))-6)-5)*4)+3)+2)-1)'

We have an answer! And in a lot less than 8 hours, thanks to dynamic programming! 

Removing unnecessary brackets, this equation is equivalent to:

In [11]:
(10 + 9 * 8 * 7 - 6 - 5) * 4 + 3 + 2 - 1

2016

Here are solutions for nearby years:

In [12]:
{y: expressions(c10)[y] for y in range(2010, 2025)}

{2010: '((((10*((9+((8+7)*6))+(5/4)))+3)*2)-1)',
 2011: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)-1)',
 2012: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)/1)',
 2013: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)+1)',
 2014: '((((10-(9*8))*(7-(6*((5+4)+3))))/2)-1)',
 2015: '((((((((10*9)+8)-7)*(6+5))+4)+3)*2)-1)',
 2016: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)-1)',
 2017: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)/1)',
 2018: '(((((((10+((9*8)*7))-6)-5)*4)+3)+2)+1)',
 2019: '((((10+9)+((8+7)*(6+((5*4)*3))))*2)+1)',
 2020: '((((10+((((9*8)*7)/(6-5))*4))-3)-2)-1)',
 2021: '((((10+((((9*8)*7)/(6-5))*4))-3)-2)/1)',
 2022: '((((10+((((9*8)*7)/(6-5))*4))-3)-2)+1)',
 2023: '(((((10+(((9*(8+7))*6)*5))/4)-3)*2)-1)',
 2024: '(((((10+(((9*(8+7))*6)*5))/4)-3)*2)/1)'}

# Counting Solutions

Alex Bellos had another challenge:  

> I was half hoping a computer scientist would let me know exactly how many solutions there are with only the four basic operations. Maybe someone will. 

As it stands, my program can't answer that question, because I only keep one expression for each value. 

Also, I'm not sure what it means to be a distinct solution. For example, are `((10+9)+8)` and `(10+(9+8))` different, or are they same, because they both are equivalent to `(10+9+8)`? Similarly, are `((3-2)-1)` and `(3-(2+1)` different, or the same because they both are equivalent to `(3 + -2 + -1)`? I think the notion of "distinct solution" is just inherently ambiguous, and each of these questions could reasonably be answered either way. My choice is to count each of these as distinct: every expression has exactly ten numbers, nine operators, and nine pairs of brackets, and if an expression differs in any character, it is different. But I won't argue with anyone who prefers a different definition of "distinct solution."

So how can I count expressions? One approach would be to go back to enumerating every equation (all 4862 &times; 4<sup>9</sup> = 1.2 bilion of them) and checking which ones equal 2016. That would take about 40 hours with my Python program, but I could get it under 40 minutes in a more efficient language.

Another approach is to count subexpressions as the table is filled in. We won't enumerate all the expressions, just count them. I'll introduce a second table, `COUNTS`, such that

    COUNTS[(10, 9, 8)][27] == 2
    
because there are 2 ways to make 27 with the numbers `(10, 9, 8)`, namely, `((10+9)+8)` and `(10+(9+8))`.
How do we compute the counts? By looking at every split and operator choice that can make the value, and summing up (over all of these) the product of the counts for the two sides.  For example, there are 2 ways to make 27 with `(10 ... 8)`, and it turns out there are 3526 ways to make 1 with `(7 ... 1)`. So there are 2 &times; 3526 = 7052 ways to make 28 with this split by adding 27 and 1. 

I'll make `expr` maintain `COUNTS` as well as `EXPS`. And I'll define  `clear` to clear out the cache of `COUNTS` and `EXPS`, so that we can fill the tables with our new, improved entries.

In [13]:
COUNTS = defaultdict(Counter) # e.g., COUNTS[(10, 9, 8)][27] == 2

def expressions(numbers):
    "Fill EXPS table for numbers, and all sub-sequences of numbers. Return EXPS[numbers]"
    if numbers in EXPS: # Already did the work
        pass
    elif len(numbers) == 1: # Only one way to make an expression out of a single number
        expr(numbers, numbers[0], str(numbers[0]), 1)
    else: # Split in all ways; fill tables for left and right; combine tables in all ways
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in itertools.product(expressions(Lnums), expressions(Rnums)):
                Lexp, Rexp = '(' + EXPS[Lnums][L], EXPS[Rnums][R] + ')'
                count = COUNTS[Lnums][L] * COUNTS[Rnums][R]
                expr(numbers, L * R, Lexp + '*' + Rexp, count)
                expr(numbers, L - R, Lexp + '-' + Rexp, count)
                expr(numbers, L + R, Lexp + '+' + Rexp, count)
                if R != 0: 
                    expr(numbers, L / R, Lexp + '/' + Rexp, count)
    return EXPS[numbers]

def expr(numbers, val, exp, count):
    "Fill EXPS[numbers][val] with exp, and increment COUNTS."
    EXPS[numbers][val] = exp
    COUNTS[numbers][val] += count
    
def clear(): EXPS.clear(); COUNTS.clear()

In [14]:
clear()
expressions((10, 9, 8))

{-62: '(10-(9*8))',
 -7: '((10-9)-8)',
 -6.888888888888889: '((10/9)-8)',
 0.125: '((10-9)/8)',
 0.1388888888888889: '((10/9)/8)',
 0.5882352941176471: '(10/(9+8))',
 2.375: '((10+9)/8)',
 8: '((10-9)*8)',
 8.875: '(10-(9/8))',
 8.88888888888889: '((10/9)*8)',
 9: '((10-9)+8)',
 9.11111111111111: '((10/9)+8)',
 10: '(10/(9-8))',
 11: '((10+9)-8)',
 11.125: '(10+(9/8))',
 11.25: '((10*9)/8)',
 27: '((10+9)+8)',
 82: '((10*9)-8)',
 98: '((10*9)+8)',
 152: '((10+9)*8)',
 170: '(10*(9+8))',
 720: '((10*9)*8)'}

In [15]:
COUNTS[(10, 9, 8)]

Counter({-62: 1,
         -7: 2,
         -6.888888888888889: 1,
         0.125: 1,
         0.1388888888888889: 2,
         0.5882352941176471: 1,
         2.375: 1,
         8: 1,
         8.875: 1,
         8.88888888888889: 2,
         9: 2,
         9.11111111111111: 1,
         10: 2,
         11: 2,
         11.125: 1,
         11.25: 2,
         27: 2,
         82: 2,
         98: 1,
         152: 1,
         170: 1,
         720: 2})

In [16]:
COUNTS[(10, 9, 8)][27]

2

Looks good to me. Now let's repeat the computation, this time keeping track of `COUNTS`:

In [17]:
clear()

%time expressions(c10)[2016]

CPU times: user 1min 4s, sys: 1.15 s, total: 1min 5s
Wall time: 1min 6s


'(((((((10+((9*8)*7))-6)-5)*4)+3)+2)-1)'

# The Answer (?)

Now we can read off the answer:

In [18]:
COUNTS[c10][2016]

30066

This says there are 30,066 distinct expressions for 2016. 

**But I don't believe it.**

Why not? Because floating point division can have round-off errors.  

# Dealing with Round-off Errors

Consider this:

In [19]:
2015 + 1/3 + 1/3 + 1/3

2015.9999999999998

In [20]:
2015 + 1/3 + 1/3 + 1/3 == 2016

False

This means there might be perfectly good expressions that are counted under `2015.9999999999998` (or some similar number) when they should be counted for `2016`. Let's find all the values that are very near to `2016`:

In [21]:
epsilon = 10 ** -8

{y for y in expressions(c10)
 if abs(y - 2016) < epsilon}

{2015.999999999997,
 2015.999999999999,
 2015.9999999999993,
 2015.9999999999995,
 2015.9999999999998,
 2016,
 2016.0000000000002,
 2016.0000000000005,
 2016.0000000000018,
 2016.000000000002,
 2016.0000000000023}

I suspect that all of these actually should be exactly 2016. 

To be absolutely sure, I could re-do the calculations using exact rational arithmetic, as provided by the `fractions.Fraction` data type. From experience I know that would be an order of magnitude slower, so instead I'll just add up all the counts in the `COUNTS` table that are within epsilon of 2016:

In [22]:
{y: COUNTS[c10][y] for y in expressions(c10)
 if abs(y - 2016) < epsilon}

{2015.999999999997: 15,
 2015.999999999999: 10,
 2015.9999999999993: 14,
 2015.9999999999995: 1930,
 2015.9999999999998: 5868,
 2016: 30066,
 2016.0000000000002: 5792,
 2016.0000000000005: 510,
 2016.0000000000018: 264,
 2016.000000000002: 18,
 2016.0000000000023: 12}

# The Answer (!)

In [23]:
sum(_.values())

44499

I have more confidence in this answer, 44,499, than in 30,066, but I wouldn't accept it as definitive until it was independently verified and passed an extensive test suite. And of course, if you have a different definition of "distinct solution," you will get a different answer.

In [ ]:
clear()

# Exponentiation, and Four 4s

Now let's turn to another of Alex's puzzles: making 2016 and other target values from a string of four or five `4`s,  with exponentiation allowed. Exponentiation is tricky for five reasons:

- **Division by zero**: `(0 ^ -1)` is the same as `(1 / 0)`, and gives a `ZeroDivisionError`.
- **Irrationals**: `(3 ^ (1 / 2))` is an irrational number; so we can't do exact rational arithmetic.
- **Imaginaries**: `(-1 ^ (1 / 2))` is an imaginary number, but Python gives a `ValueError`.
- **Overflow**: `(10. ^ (9. ^ 8.))`, as a `float`, gives a `OverflowError`.
- **Finite memory**: [`(10 ^ (9 ^ (8 * 7)))`](http://www.wolframalpha.com/input/?i=10+%5E+9+%5E+56), as an `int`, gives an `OutOfMemoryError` (even if your memory uses every atom on Earth).

How do we deal with this? We can't do exact rational arithmetic. We could try to do exact *algebra*, perhaps using [SymPy](http://www.sympy.org/en/index.html), but that seems difficult
and computationally expensive, so instead I will abandon the goal of exact computation, and do everything in the domain of floats (reluctantly accepting that there will be some round-off errors). We'll coerce numbers to floats when we first put them in the table, and all subsequent operations will be with floats. I define a new function, `expr2`,  to call `expr`, catching arithmetic errors. Since we are making some rather arbitrary decisions about what expressions are allowed (e.g. imaginary numbers are not), I'll give up on trying to maintain `COUNTS`.

In [62]:
from operator import add, sub, mul, truediv

def expressions(numbers):
    "Fill EXPS table for numbers, and all sub-sequences of numbers. Return EXPS[numbers]"
    if numbers in EXPS: # Already did the work
        pass
    elif len(numbers) == 1: # Only one way to make an expression out of a single number
        expr(numbers, float(numbers[0]), str(numbers[0]))
    else: # Split in all ways; fill tables for left and right; combine tables in all ways
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in itertools.product(expressions(Lnums), expressions(Rnums)):
                Lexp, Rexp = '(' + EXPS[Lnums][L], EXPS[Rnums][R] + ')'
                expr2(numbers, L, pow,     R, Lexp + '^' + Rexp)
                expr2(numbers, L, truediv, R, Lexp + '/'  + Rexp)
                expr2(numbers, L, mul,     R, Lexp + '*'  + Rexp)
                expr2(numbers, L, add,     R, Lexp + '+'  + Rexp)
                expr2(numbers, L, sub,     R, Lexp + '-'  + Rexp)
    return EXPS[numbers]

def expr2(numbers, L, op, R, exp): 
    "Fill table entries for op(L, R), catching errors."
    try:
        expr(numbers, op(L, R), exp)
    except (ArithmeticError, ValueError):
        pass
    
def expr(numbers, value, exp): EXPS[numbers][value] = exp

Now we can solve the "2016 with five fours" puzzle:

In [64]:
clear()

expressions((4, 4, 4, 4, 4))[2016]

'(((4^4)-4)*(4+4))'

I'll define a function to create a table of makeable integers:

In [26]:
def makeable(numbers):
    "A table of {i: expression} for all integers i from 0 up to first unmakeable."
    return {i: expressions(numbers)[i]
            for i in range(unmakeable(numbers))}

def unmakeable(numbers):
    "Smallest positive integer than can't be made by numbers."
    return next(i for i in itertools.count(1) if i not in expressions(numbers))

We'll use this to see if we can solve the "0 to 9 with four fours" puzzle:

In [65]:
makeable((4, 4, 4, 4))

{0: '(((4+4)-4)-4)',
 1: '(((4+4)-4)/4)',
 2: '((4/(4+4))*4)',
 3: '(((4+4)+4)/4)',
 4: '(((4-4)*4)+4)',
 5: '(((4*4)+4)/4)',
 6: '(((4+4)/4)+4)',
 7: '((4-(4/4))+4)',
 8: '(((4+4)-4)+4)',
 9: '(((4/4)+4)+4)'}

Yes: we can get 0 to 9 (but not 10).

Now I'll see what integers we can make with five fives. Legend has it that you can get all the way up to 55:

In [67]:
clear()

five5s = (5, 5, 5, 5, 5)

makeable(five5s)

{0: '((((5-5)*5)-5)+5)',
 1: '(((5-(5*5))/5)+5)',
 2: '((((5+5)/5)-5)+5)',
 3: '((((5*5)-5)-5)/5)',
 4: '((((5-5)-5)/5)+5)',
 5: '((5/((5^5)^5))+5)',
 6: '((((5/5)+5)*5)/5)',
 7: '(((5/5)+(5/5))+5)',
 8: '((((5+5)+5)/5)+5)',
 9: '((((5+5)*5)-5)/5)',
 10: '((((5*5)-5)-5)-5)',
 11: '((((5+5)*5)+5)/5)',
 12: '((((5+5)/5)+5)+5)'}

We didn't get there. 

# More Operations

With some research, I [see](http://www.infobarrel.com/Five_Fives_Problem_Recreational_Mathematics) that others who got up to 55 with five 5s used some or all of these three concepts:

- **digit concatenation**: `55`
- **decimal point**: `.5`
- **unary operations**: `-5`, `5!`, and  &radic; `5`


We'll refactor `expressions` to call the following three new subfunctions:

- `digit_expressions`: For every subsequence of numbers, we'll smush the digits together, and then make a table entry for those resulting digits as an integer, and with a decimal point in each possible position.
- `binary_expressions`: The code that previously was the main body of `expressions`.
- `unary_expressions`: Apply the unary operators to every entry in the table. 

We'll still do all computation in the domain of floats.

In [68]:
from math import sqrt, factorial

def expressions(numbers):
    "Fill EXPS table for numbers, and all sub-sequences of numbers. Return EXPS[numbers]"
    if numbers not in EXPS: 
        digit_expressions(numbers)
        binary_expressions(numbers)
        unary_expressions(numbers)
    return EXPS[numbers]

def digit_expressions(numbers):
    "Fill EXPS with digits, with opyional minus sign and decimal point."
    # digit_expressions(1, 2) => 12, .12, 1.2
    exp = ''.join(str(n) for n in numbers)
    for d in range(len(exp) + 1):
        decimal = (exp[:d] + '.' + exp[d:]).rstrip('.')
        if d < 2 or not decimal.startswith('0'): # '01' is illegal; '0', '0.1' ok
            expr(numbers, float(decimal), decimal)
            expr(numbers, -float(decimal), '-' + decimal)
            
def binary_expressions(numbers):
    "Fill EXPS with all expressions formed by splitting numbers and combining with an op."
    for (Lnums, Rnums) in splits(numbers):
        for (L, R) in itertools.product(expressions(Lnums), expressions(Rnums)):
            Lexp, Rexp = '(' + EXPS[Lnums][L], EXPS[Rnums][R] + ')'
            expr2(numbers, L, truediv, R, Lexp + '/'  + Rexp)
            expr2(numbers, L, mul,     R, Lexp + '*'  + Rexp)
            expr2(numbers, L, add,     R, Lexp + '+'  + Rexp)
            expr2(numbers, L, sub,     R, Lexp + '-'  + Rexp)
            if 1 <= R <= 10 and (L > 0 or int(R) == R):
                expr2(numbers, L, pow, R, Lexp + '^' + Rexp)
                
def unary_expressions(numbers):
    "Fill tables for -v, √v and v!"
    for v in tuple(EXPS[numbers]):
        exp = EXPS[numbers][v]
        expr(numbers, -v, '-(' + exp + ')')
        if v > 0: 
            expr(numbers, sqrt(v), '√' + exp)
        if 3 <= v <= 6 and v == int(v):
            expr(numbers, factorial(v), exp + '!')

Now that we have more variety in the types of expressions formed, I want to choose a "good" expression to represent each value. I'll modify `expr` so that when there are multiple expressions for a value, it chooses the one with the least "weight," where I define the `weight` of a string as the sum of the weights of the characters, where the square root sign is the heaviest, and other characters are as listed.

In [69]:
def expr(numbers, value, exp): 
    "Fill EXPS[numbers][val] with exp, unless we already have a lighter exp."
    if value not in EXPS[numbers] or weight(exp) <= weight(EXPS[numbers][value]):
        EXPS[numbers][value] = exp
        
WEIGHTS = {'√':3, '.':2, '^':1.3, '/':1.2, '-':1.1}
        
def weight(exp, weights=WEIGHTS): 
    return sum(weights.get(c, 1) for c in exp)

We'll try again:

In [70]:
clear()

%time makeable(five5s)

CPU times: user 6min 4s, sys: 2.81 s, total: 6min 7s
Wall time: 6min 7s


{0: '((55-55)*5)',
 1: '((55/55)^5)',
 2: '(55/(55*.5))',
 3: '√(5!-(555/5))',
 4: '(5-(55/55))',
 5: '((55-55)+5)',
 6: '((55/55)+5)',
 7: '(((55+5)/5)-5)',
 8: '(((5!-55)/5)-5)',
 9: '(5!-(555/5))',
 10: '(5!-(55+55))',
 11: '((.55*5!)-55)',
 12: '((5.5/55)*5!)',
 13: '(((55+5)+5)/5)',
 14: '((5*5)-(55/5))',
 15: '((55/5.5)+5)',
 16: '((55+(5*5))/5)',
 17: '(((55+5)/5)+5)',
 18: '(((5!-55)/5)+5)',
 19: '((5*5)-((5/5)+5))',
 20: '(55/(.55*5))',
 21: '(((55/5)+5)+5)',
 22: '((55+55)/5)',
 23: '(((55/.5)+5)/5)',
 24: '(5-(55/55))!',
 25: '(55-((5*5)+5))',
 26: '(55-((5!/5)+5))',
 27: '((55/(5*.5))+5)',
 28: '((55+(5/5))*.5)',
 29: '(((5*5)-(5/5))+5)',
 30: '(((55/5)-5)*5)',
 31: '(((55*5)-5!)/5)',
 32: '(((5-5!)/5)+55)',
 33: '((55+5)*.55)',
 34: '(((55+5!)-5)/5)',
 35: '((55-(5*5))+5)',
 36: '((55/5)+(5*5))',
 37: '(((5!-55)+5!)/5)',
 38: '(((.55*.5)*5!)+5)',
 39: '((((5!/5)+5)+5)+5)',
 40: '(55-((5+5)+5))',
 41: '(5!-(55+(5!/5)))',
 42: '((.55*5!)-(5!/5))',
 43: '(55-(5!/(5+5)))',
 44

Wow! We almost tripled the 55 goal! (Although the increased number of expressions means it took 6 minutes.)

I have to say, I would never have come up with the solution for 81 on my own. It works because (√(5 - .5) \* √.5) = √(4.5 \* 0.5) = √(9/4) = 3/2.

# Even More Operations 

At the risk of making the computation take even longer, I'm going to add two more unary operations:

- **Floor**: &lfloor;*x*&rfloor; is the largest integer less than or equal to *x* (in other words, rounding down).
- **Ceiling**: &lceil;*x*&rceil; is the smallest integer greater than or equal to *x* (in other words, rounding up).

These operations are useful because they produce integers, and our targets are the integers (from 0 up to whatever). 

In addition, I'll allow two consecutive applications of unary operators, thus allowing expressions such as

- ⌊√5⌋ = 2
- ⌈5.5⌉! = 720

But still not allowing three consecutive applications, such as

- ⌈√5⌉! = 6



In [72]:
from math import floor, ceil

floor_ceil_allowed = True # Should we allow floor and ceil?

def unary_expressions(numbers):
    "Fill tables for -v, √v and v!, ⌊x⌋, and ⌈x⌉"
    for i in range(2):
        for v in tuple(EXPS[numbers]):
            exp = EXPS[numbers][v]
            expr(numbers, -v, '-(' + exp + ')')
            if 0 < v <= 100 and 4*v == round(4*v):
                # Be stingier in allowing √ operator
                expr(numbers, sqrt(v), '√' + exp)
            if 3 <= v <= 6 and v == int(v):
                expr(numbers, factorial(v), exp + '!')
            if floor_ceil_allowed and v != int(v):
                uexp = unbracket(exp)
                expr(numbers, floor(v), '⌊' + uexp + '⌋')
                expr(numbers, ceil(v),  '⌈' + uexp + '⌉')
            
def unbracket(exp):
    "Remove outer brackets from exp if they are there."
    if exp.startswith('(') and exp.endswith(')'):
        return exp[1:-1]
    else:
        return exp

Let's try (warning&mdash;it will take 20 or 30 minutes to run):

In [73]:
clear()

%time makeable(five5s)

CPU times: user 24min 17s, sys: 3.22 s, total: 24min 21s
Wall time: 24min 23s


{0: '⌊55/555⌋',
 1: '⌈55/555⌉',
 2: '⌊.5555*5⌋',
 3: '⌈.5555*5⌉',
 4: '⌊5-.5555⌋',
 5: '⌊5.5555⌋',
 6: '⌈5.5555⌉',
 7: '⌈5.55+.55⌉',
 8: '⌈5.555+√5⌉',
 9: '⌊55/5.55⌋',
 10: '⌊555/55⌋',
 11: '⌈555/55⌉',
 12: '⌈55.55/5⌉',
 13: '⌊(.555*5)*5⌋',
 14: '⌈(.555*5)*5⌉',
 15: '⌊(5.55+5)+5⌋',
 16: '⌈(5.55+5)+5⌉',
 17: '⌈(55.5/5)+5⌉',
 18: '⌊55/⌈5!/55⌉⌋',
 19: '⌈55/⌈5!/55⌉⌉',
 20: '(⌊555/5!⌋*5)',
 21: '⌊5!/5.555⌋',
 22: '⌈5!/5.555⌉',
 23: '⌈555/(5*5)⌉',
 24: '⌊5-.5555⌋!',
 25: '(⌊5.555⌋*5)',
 26: '⌈(55*55)/5!⌉',
 27: '⌊5.555*5⌋',
 28: '⌈5.555*5⌉',
 29: '(⌊555/5!⌋!+5)',
 30: '⌊.555*55⌋',
 31: '⌈.555*55⌉',
 32: '⌊(5.55*5)+5⌋',
 33: '⌈(5.55*5)+5⌉',
 34: '⌈5.55*⌈5.5⌉⌉',
 35: '⌊(55*.55)+5⌋',
 36: '⌈(55*.55)+5⌉',
 37: '⌊(55/√55)*5⌋',
 38: '⌈(55/√55)*5⌉',
 39: '⌊55.55*√.5⌋',
 40: '(55-((5+5)+5))',
 41: '⌊5.55*√55⌋',
 42: '⌈5.55*√55⌉',
 43: '⌈55-(√55+5)⌉',
 44: '(55-(55/5))',
 45: '((5!*5)-555)',
 46: '⌊5555/5!⌋',
 47: '⌈5555/5!⌉',
 48: '⌊55.5-√55⌋',
 49: '⌊55-5.55⌋',
 50: '⌈55-5.55⌉',
 51: '⌈55.55-5⌉',
 

The output got truncated by Jupyter Notebook after 1000 entries. Let's see what the first unmakeable integer actually is:

In [74]:
unmakeable(five5s)

23308

I think we've conquered the five 5s problem!


# Countdown to 2018

On January 1 2018, [Michael Littman](http://cs.brown.edu/~mlittman/) posted this:

> `2+0+1x8, 2+0-1+8, (2+0-1)x8, |2-0-1-8|, -2-0+1x8, -(2+0+1-8), sqrt(|2+0-18|), 2+0+1^8, 20-18, 2^(0x18), 2x0x1x8`... Happy New Year!

Can we replicate that countdown, using the four digits of the year, in order, with operators and parens to evaluate each of the numbers from 10 to 1? I'm assuming Michael is disallowing floor and ceil, but allowing our other operators.

In [75]:
clear()
floor_ceil_allowed = False

def littman_countdown(year):
    "Return a Littman countdown for the year."
    return ', '.join(littman(year, i) for i in range(10, -1, -1))

def littman(year, i):
    "Return a string that makes i with the digits of year."
    digits = tuple(map(int, str(year)))
    return unbracket(expressions(digits).get(i, '???'))

littman_countdown(2018)

'(20*.1)+8, (2.0-1)+8, (2.0-1)*8, (-2.0+1)+8, √(20*1.8), (-2.0-1)+8, √(-2.0+18), 2.0+(1^8), 20-18, (2.0-1)^8, (2*0)*18'

Similar results, with some alternatives for some numbers.

Let's look at the decade:

In [76]:
for y in range(2011, 2020):
    print('{} ... Happy New Year {}!'.format(littman_countdown(y), y))

20/(1+1), 20-11, (-.20+1)/.1, (2.0+1)!+1, √(20-11)!, (2.0+1)!-1, (2.0+1)+1, √(20-11), 2+(0*11), (2.0-1)*1, (2*0)*11 ... Happy New Year 2011!
-2.0+12, (2.0+1)^2, 20-12, 2.0+(1/.2), (2.0+1)*2, (2.0+1)+2, (20*1)*.2, (2.0-1)+2, 2+(0*12), (2.0+1)-2, (2*0)*12 ... Happy New Year 2012!
20/(-1+3), (2.0+1)*3, 2.0*(1+3), 20-13, (20*1)*.3, 20/(1+3), (2.0-1)+3, (2.0-1)*3, 2+(0*13), (2.0+1)/3, (2*0)*13 ... Happy New Year 2013!
2.0*(1+4), √((2.0+1)^4), (20*1)*.4, (2.0+1)+4, 20-14, (20*1)/4, 20/(1+4), (-20-1)+4!, 2+(0*14), .20*(1+4), (2*0)*14 ... Happy New Year 2014!
(20*1)*.5, √(201-5!), (2.0+1)+5, (20*.1)+5, (20*.15)!, 20-15, (20*1)/5, 20*.15, 2+(0*15), (.20*1)*5, (2*0)*15 ... Happy New Year 2015!
2.0*(-1+6), (2.0+1)+6, (20*.1)+6, (2.0-1)+6, √(20+16), 20/√16, 20-16, 2.0+(1^6), 2+(0*16), (2.0-1)^6, (2*0)*16 ... Happy New Year 2016!
(2.0+1)+7, (20*.1)+7, (2.0-1)+7, (2.0-1)*7, (20-17)!, (-2.0*1)+7, (-2.0-1)+7, 20-17, 2+(0*17), (2.0-1)^7, (2*0)*17 ... Happy New Year 2017!
(20*.1)+8, (2.0-1)+8, (2.0-1)*8